# Demo No. 1: Hello binary!


This set of tutorials will guide you trough the basic structure of this package. This particular demo will show you how to build your binary system and access its basic parameters.

## Import correct modules

Objects modelled in our first binary system are divided into two groups:
* system - binary system,
* compoents - stars.

Each of those object types have to be imported separatelly using the following imports:   

In [1]:
from elisa import BinarySystem
from elisa import Star

/home/cepheus/elisa_jup/lib/python3.6/site-packages/elisa/conf/settings.py:397: DeprecationWarning: Variable `atlas` in configuration section `support` is not longer supported and will be removed in future version.
Use atmosphere definition as initial parameter for given celestial object
  "for given celestial object", DeprecationWarning)
/home/cepheus/elisa_jup/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Setting up the components

Now we can initialize components with their respective parameters. Values of gravity farkening exponents, albedos and metalicities are optional and the default values will be assigned to them in case of their absence. Gravity darkening and bolometric albedo are interpolated based on Figure 6 in Claret (2003) and Figure 2 in Claret (2001). Metallicity is kept fixed on solar values by default. Lets define the primary and secondary component:

In [2]:
from astropy import units as u

primary = Star(
    mass=2.15 * u.solMass,
    surface_potential=3.6,
    synchronicity=1.0,
    t_eff=10000 * u.K,
    gravity_darkening=1.0, # optional argument, otherwise interpolated
    discretization_factor=5,  # angular size (in degrees) of the surface elements, optional
    albedo=0.6,  # optional argument, otherwise interpolated
    metallicity=0.0  # optional argument  
)

secondary = Star(
    mass=0.45 * u.solMass,
    surface_potential=5.39,
    synchronicity=1.0,
    t_eff=6500 * u.K,
)

Component parameters can be also set and modified separatelly using setters:

In [3]:
primary.synchronicity = 1.0

and accesed via getters:

In [4]:
primary.synchronicity

1.0

As soon as the components are defined, we can set up BinarySystem instance:

In [5]:
bs = BinarySystem(
    primary=primary,
    secondary=secondary,
    argument_of_periastron=58 * u.deg,
    gamma=-30.7 * u.km / u.s,
    period=2.5 * u.d,
    eccentricity=0.0,
    inclination=85 * u.deg,
    primary_minimum_time=2440000.00000 * u.d,
    phase_shift=0.0,
)


2022-01-24 09:44:30,860 - 12229 - binary_system.system - INFO: initialising object BinarySystem
2022-01-24 09:44:31,014 - 12229 - binary_system.system - INFO: setting discretization factor of secondary component to 8.00 according to discretization factor of the companion.


where again, parameters can be accesed and overwriten using getters and setters

In [6]:
bs.eccentricity = 0.1
bs.eccentricity

0.1

Changing of any system or stellar parameters require re-initialization of the `BinarySystem`. Re-initialization also makes sure that the system with new parameters is physically valid and can be built. Re-initialization of the binary system instance `bs` can be done in the following manner way:

In [7]:
bs.init()

2022-01-24 09:44:31,044 - 12229 - binary_system.system - INFO: initialising object BinarySystem
2022-01-24 09:44:31,162 - 12229 - astropy - WARNING: AstropyDeprecationWarning: The truth value of a Quantity is ambiguous. In the future this will raise a ValueError.


We can also have a look on the interolated gravity darkening factor and albedo on the secondary component:

In [8]:
bs.secondary.albedo, bs.secondary.gravity_darkening

(0.7126008613327499, 0.5302088904333047)

In [9]:
bs.default_internal_units

## Accesing parameters

Finally, after binary system initialization, we can acces basic properties of our system such as semi-major axis,  morphology, and radii:

In [10]:
bs.semi_major_axis*bs.default_internal_units.system.semi_major_axis.to(u.solRad)  # conversion to solar radii

10.659024832175282

morphology of the system can be also displayed:

In [11]:
bs.morphology

'detached'

along with radii of the components in default SMA (semi major axis) units:

In [12]:
print(f'Polar radius: {primary.polar_radius}')
print(f'Side radius: {primary.side_radius}')
print(f'Backward radius: {primary.backward_radius}')
print(f'Forward radius: {primary.forward_radius}')
print(f'Equivalent radius: {primary.equivalent_radius}')

Polar radius: 0.29391569123711786
Side radius: 0.29861793999676073
Backward radius: 0.30153964149907264
Forward radius: 0.30346500951048827
Equivalent radius: 0.29801981218086737


Values for critical potentials for each component can be accessed using these getters:

In [13]:
print(f'Critical potential for primary component: {primary.critical_surface_potential}')
print(f'Critical potential for secondary component: {secondary.critical_surface_potential}')

Critical potential for primary component: 2.4269724166520685
Critical potential for secondary component: 2.440275836628986


*Note that critical potential values are not the same due to eccentricity is set on value 0.1. Those values are critical potentials during periastron passage.

## Initializing binary system from a dictionary (json)

Instances of a `BinarySystem` can be also initialized using a dictionary of parameters that can be stored in a json format. This approach enables to define a system using different combination of parameters. Currently, the `standard` and `community` format are available. To achieve this, we can use `BinarySystem.from_json()` function:

In [14]:
standard_params = {
    "system": {
        "inclination": 90.0,
        "period": 10.1,
        "argument_of_periastron": "90.0 deg",  # string representation of astropy quantity is also valid
        "gamma": 0.0,
        "eccentricity": 0.3,
        "primary_minimum_time": 0.0,
        "phase_shift": 0.0
    },
    "primary": {
        "mass": "4.0e30 kg",
        "surface_potential": 7.1,
        "synchronicity": 1.0,
        "t_eff": 6500.0,
        "gravity_darkening": 1.0,
        "discretization_factor": 5,
        "albedo": 1.0,
        "metallicity": 0.0
    },
    "secondary": {
        "mass": 2.0,  # default unit is solMass"
        "surface_potential": 7.1,
        "synchronicity": 1.0,
        "t_eff": 6500.0,
        "gravity_darkening": 1.0,
        "discretization_factor": 5,
        "albedo": 1.0,
        "metallicity": 0.0
    }
}
standard_binary = BinarySystem.from_json(standard_params)

2022-01-24 09:44:31,300 - 12229 - binary_system.system - INFO: initialising object BinarySystem


`standard`format of parameters requires masses of the components, whereas the `community` format requires parameters `mass_ratio` and `semi_major_axis` instead to define a system:

In [15]:
community_params = {
    "system": {
        "inclination": 90.0,
        "period": 10.1,
        "argument_of_periastron": 90.0,
        "gamma": 0.0,
        "eccentricity": 0.3,
        "primary_minimum_time": 0.0,
        "phase_shift": 0.0,
        "semi_major_axis": 10.5,  # default unit is solRad
        "mass_ratio": 0.5
    },
    "primary": {
        "surface_potential": 7.1,
        "synchronicity": 1.0,
        "t_eff": "6500.0 K",
        "gravity_darkening": 1.0,
        "discretization_factor": 5,
        "albedo": 1.0,
        "metallicity": 0.0
    },
    "secondary": {
        "surface_potential": 7.1,
        "synchronicity": 1.0,
        "t_eff": 6500.0,
        "gravity_darkening": 1.0,
        "discretization_factor": 5,
        "albedo": 1.0,
        "metallicity": 0.0
    }
}

community_binary = BinarySystem.from_json(community_params)

2022-01-24 09:44:31,449 - 12229 - binary_system.system - INFO: initialising object BinarySystem


## Acessing units of binary system parameters

To avoid mass confusion, ELISa uses default units to perform calculations in the backend. Module `elisa.units` stores deafult units that ELISa uses internally. Eg.:

In [16]:
from elisa import units

units.DISTANCE_UNIT, units.PERIOD_UNIT

(Unit("m"), Unit("d"))

Additionally, internal units of each binary system parameter can be acessed:

In [17]:
bs.default_internal_units.system.inclination, bs.default_internal_units.primary.mass

(Unit("rad"), Unit("kg"))

The set of whole set of parameters can be acessed at once:

In [18]:
bs.default_internal_units.as_dict()

{'system': {'inclination': 'rad',
  'period': 'd',
  'eccentricity': '',
  'argument_of_periastron': 'rad',
  'gamma': 'm / s',
  'phase_shift': '',
  'additional_light': '',
  'primary_minimum_time': 'd',
  'semi_major_axis': 'm'},
 'primary': {'mass': 'kg',
  't_eff': 'K',
  'surface_potential': '',
  'synchronicity': '',
  'metallicity': '',
  'gravity_darkening': '',
  'albedo': '',
  'discretization_factor': 'rad',
  'spots': {'longitude': 'rad',
   'latitude': 'rad',
   'angular_radius': 'rad',
   'temperature_factor': '',
   'discretization_factor': 'rad'},
  'pulsations': {'l': '',
   'm': '',
   'amplitude': 'm / s',
   'frequency': 'Hz',
   'start_phase': 'rad',
   'mode_axis_theta': 'rad',
   'mode_axis_phi': 'rad',
   'temperature_perturbation_phase_shift': 'rad',
   'horizontal_to_radial_amplitude_ratio': '',
   'temperature_amplitude_factor': '',
   'tidally_locked': "<class 'bool'>"}},
 'secondary': {'mass': 'kg',
  't_eff': 'K',
  'surface_potential': '',
  'synchronici

However, internal binary system units are not ideal for the user to define binary system parameters. Therefore, default input units of the BinarySystem instance (if unit is not specified) can be accessed in similar fashion:

In [19]:
bs.default_input_units.system.inclination, bs.default_input_units.primary.mass

(Unit("deg"), Unit("solMass"))

Here is the set of all input units:

In [20]:
bs.default_input_units.as_dict()

{'system': {'inclination': 'deg',
  'period': 'd',
  'eccentricity': '',
  'argument_of_periastron': 'deg',
  'gamma': 'm / s',
  'phase_shift': '',
  'additional_light': '',
  'primary_minimum_time': 'd',
  'semi_major_axis': 'solRad'},
 'primary': {'mass': 'solMass',
  't_eff': 'K',
  'surface_potential': '',
  'synchronicity': '',
  'metallicity': '',
  'gravity_darkening': '',
  'albedo': '',
  'discretization_factor': 'deg',
  'spots': {'longitude': 'deg',
   'latitude': 'deg',
   'angular_radius': 'deg',
   'temperature_factor': '',
   'discretization_factor': 'deg'},
  'pulsations': {'l': '',
   'm': '',
   'amplitude': 'm / s',
   'frequency': '1 / d',
   'start_phase': 'rad',
   'mode_axis_theta': 'deg',
   'mode_axis_phi': 'deg',
   'temperature_perturbation_phase_shift': 'rad',
   'horizontal_to_radial_amplitude_ratio': '',
   'temperature_amplitude_factor': '',
   'tidally_locked': "<class 'bool'>"}},
 'secondary': {'mass': 'solMass',
  't_eff': 'K',
  'surface_potential': 

and finally, a set of only system input units:

In [21]:
bs.default_input_units.system.as_dict()

{'inclination': 'deg',
 'period': 'd',
 'eccentricity': '',
 'argument_of_periastron': 'deg',
 'gamma': 'm / s',
 'phase_shift': '',
 'additional_light': '',
 'primary_minimum_time': 'd',
 'semi_major_axis': 'solRad'}

## References:

- Claret, A. 2001, MNRAS. 327, 989–994
- Claret, A. 2003, A&A 406, 623–628